# Cleanup on Row 7

1. Clean the File
2. Handle Corrupt Rows
3. Explore Data Set Bias

## Import Libraries

In [54]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt

### Helper Display Functions

In [3]:
def display_df(df, nrows=10, ncols=None):
    with pd.option_context('display.max_rows', nrows, 'display.max_columns', ncols):
        display (df)
def print_row(df, row):
    for ctr,i in enumerate(df.iloc[row]):
        print (str(df.columns[ctr])+": "+str(i))

### Helper Statistical Functions

In [4]:
#gets ratio of NaNs for each column
def stats_NaN(df):
    df_stats = pd.DataFrame(index=[df.columns], columns=["NaN Ratio"])
    for col in df.columns:
        df_stats["NaN Ratio"][col] = df[col].isna().sum()/len(df) #NaN ratio
    return df_stats.sort_values(by=['NaN Ratio'])

#helps see what fields may be incorrect by having more values than should be
def stats_unique(df, labels):
    df_counts = pd.DataFrame(index=labels,columns=['Unique Count'])
    for l in labels:
        df_counts['Unique Count'][l] = df[l].nunique()
    return df_counts

## Importing the Data

In [5]:
df_raw = pd.read_csv("../dirty_sample_small.csv")

ParserError: Error tokenizing data. C error: Expected 48 fields in line 652242, saw 49


Some rows have an extra field. We shall investigate what this is. First we will import the file in a more managable way, capable of handling these erroneous lines.

In [10]:
df_raw = pd.DataFrame([line.strip().split(',') for line in open('../dirty_sample_small.csv', 
                                                                'r', encoding="ISO-8859-1")])

In [11]:
len(df_raw)

661487

## Investigating Corrupt Lines

In [12]:
stats_NaN(df_raw)

,NaN Ratio
0,0
1,0
2,0
3,0.000270602
4,0.000498876
5,0.000757384
6,0.00102496
7,0.00126231
8,0.00155407
9,0.00181712


We see that that the extra has 98.6% field NaNs. This is enough to drop it. However we want to do our due diligence and investigate what the values that are not NaN are. Our suspicion is that it contains the empty string "". So we will query this dataset on the extra field for values that are not NaN or not the empty string. These rows with the empty string are likely corrupt.

In [16]:
df_raw[(pd.notna(df_raw[48]) & (df_raw[48] != ''))][48]

Series([], Name: 48, dtype: object)

We see that this returns nothing. Therefore everything in the extra field is either NaN or the empty string, so there is absolutely no value in keeping it and we will drop it. But first, lets see how many of these corrupt erroneous lines there are. We know that the construction of df_raw line by line as we did above is indifferent to the amount of columns in each row. Most of the time it saw 48 columns, in the later rows it saw 49 because of the extra empty string. The construction of the data set put NaNs in the earlier rows to ensure the shape of the dataset is intact (a regular rectangle). Therefore the source of the corrupt lines is the empty string, so if we count the empty strings in the extra field we will have successfully counted the amount of corrupt lines.

In [17]:
len(df_raw[((df_raw[48] == ''))][48])

9160

We see that there are 9160 of these rows.

In [18]:
display_df(df_raw[((df_raw[48] == ''))])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
652241,HarvardX/CS50x/2014_T1,1818882,True,True,True,False,False,173.76.227.129,US,United States,Americas,Newton,MA,Massachusetts,02458,Northern America,Developed regions,,,42.3533,-71.1883,el,2002,m,0.0,0.6,2013-12-04 04:02:20,2013-12-04 04:02:20.199004,2014-04-01 23:30:47.988143,2194,39,680,8,,,,,,,Student,0,0,0,honor,1,2014-06-06 16:36:03,2014-09-15 17:57:23,notpassing,
652242,HarvardX/CS50x/2014_T1,1818882,True,True,True,False,False,173.76.227.129,US,United States,Americas,Newton,MA,Massachusetts,02458,Northern America,Developed regions,,,42.3533,-71.1883,el,2002,m,0.0,0.6,2013-12-04 04:02:20,2013-12-04 04:02:20.199004,2014-04-01 23:30:47.988143,2194,39,680,8,,,,,,,Student,0,0,0,honor,1,2014-06-06 16:36:03,2014-09-15 17:57:23,notpassing,
652243,HarvardX/CS50x/2014_T1,1818882,True,True,True,False,False,173.76.227.129,US,United States,Americas,Newton,MA,Massachusetts,02458,Northern America,Developed regions,,,42.3533,-71.1883,el,2002,m,0.0,0.6,2013-12-04 04:02:20,2013-12-04 04:02:20.199004,2014-04-01 23:30:47.988143,2194,39,680,8,,,,,,,Student,0,0,0,honor,1,2014-06-06 16:36:03,2014-09-15 17:57:23,notpassing,
652244,HarvardX/CS50x/2014_T1,1818882,True,True,True,False,False,173.76.227.129,US,United States,Americas,Newton,MA,Massachusetts,02458,Northern America,Developed regions,,,42.3533,-71.1883,el,2002,m,0.0,0.6,2013-12-04 04:02:20,2013-12-04 04:02:20.199004,2014-04-01 23:30:47.988143,2194,39,680,8,,,,,,,Student,0,0,0,honor,1,2014-06-06 16:36:03,2014-09-15 17:57:23,notpassing,
652245,HarvardX/CS50x/2014_T1,1818882,True,True,True,False,False,173.76.227.129,US,United States,Americas,Newton,MA,Massachusetts,02458,Northern America,Developed regions,,,42.3533,-71.1883,el,2002,m,0.0,0.6,2013-12-04 04:02:20,2013-12-04 04:02:20.199004,2014-04-01 23:30:47.988143,2194,39,680,8,,,,,,,Student,0,0,0,honor,1,2014-06-06 16:36:03,2014-09-15 17:57:23,notpassing,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661482,HarvardX/CS50x/2014_T1,1934738,True,False,,False,False,71.41.229.179,US,United States,Americas,Dallas,TX,Texas,75201,Northern America,Developed regions,,,32.7831,-96.8067,p,1972,m,0.0,0.6,2013-07-09 01:46:41,,,,,,,,,,,,,Student,,,,honor,1,2014-06-06 15:08:24,2014-09-15 16:35:38,notpassing,
661483,HarvardX/CS50x/2014_T1,1934738,True,False,,False,False,71.41.229.179,US,United States,Americas,Dallas,TX,Texas,75201,Northern America,Developed regions,,,32.7831,-96.8067,p,1972,m,0.0,0.6,2013-07-09 01:46:41,,,,,,,,,,,,,Student,,,,honor,1,2014-06-06 15:08:24,2014-09-15 16:35:38,notpassing,
661484,HarvardX/CS50x/2014_T1,1934738,True,False,,False,False,71.41.229.179,US,United States,Americas,Dallas,TX,Texas,75201,Northern America,Developed regions,,,32.7831,-96.8067,p,1972,m,0.0,0.6,2013-07-09 01:46:41,,,,,,,,,,,,,Student,,,,honor,1,2014-06-06 15:08:24,2014-09-15 16:35:38,notpassing,
661485,HarvardX/CS50x/2014_T1,1934738,True,False,,False,False,71.41.229.179,US,United States,Americas,Dallas,TX,Texas,75201,Northern America,Developed regions,,,32.7831,-96.8067,p,1972,m,0.0,0.6,2013-07-09 01:46:41,,,,,,,,,,,,,Student,,,,honor,1,2014-06-06 15:08:24,2014-09-15 16:35:38,notpassing,


Many of these "corrupt" lines contains valid data, so we will have to handle them. It does appear though that many of these 9160 corrupt lines are indeed duplicates. Lets see how many.

In [19]:
len(df_raw[((df_raw[48] == ''))].drop_duplicates(inplace=False))

710

So of the 9160 corrupt lines, at least 9160 - 710 = 8450 are duplicates. I say at least because even those 710 lines may be duplicates with the rest of the raw dataset as this analysis above only considers rows with the empty string in the extra field. So we can analyze overall the amount of duplicates in the whole raw dataset and compare.

In [20]:
len(df_raw.drop_duplicates(inplace=False))

57223

Interestingly, there are still many more duplicates. However, if we had dropped duplicates at this stage, then we would have also gotten rid of at least 8450 corrupt lines. Therefore we posit, that dropping duplicates would drastically change the count of corrupt lines. We shall prove this with the code below.

In [21]:
len(df_raw.drop_duplicates(inplace=False)[df_raw[48] == ''])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


710

We see that that these corrupt lines do have 710 valid samples. Therefore simply dropping duplicates in the whole dataset is guarunteed to drop 8450 of the corrupt lines.

## Cleaning the Mixed Up Data

We observe that some of the data appears scrambled. Meaning some of the data for some columns is in the wrong columns. We elucidate below. But first, lets give our dataset acceptable column names and drop the erroneous column. From now on, we will be using the goal dataset (the future fully cleaned one) called `df_clean`.

In [132]:
df_clean = df_raw.copy()
df_clean = df_clean.rename(columns=df_raw.iloc[0])
df_clean = df_clean.drop(df_clean.index[0])
df_clean = df_clean.drop(np.nan, axis=1)

In [23]:
stats_unique(df_clean, df_clean.columns)

,Unique Count
course_id,21
user_id,49144
registered,3
viewed,3
explored,3
certified,2
completed,44101
ip,799
cc_by_ip,295
countryLabel,96


In [24]:
stats_NaN(df_clean)

,NaN Ratio
course_id,0
user_id,0
registered,0
viewed,0.000270603
explored,0.000498877
certified,0.000757386
completed,0.00102497
ip,0.00126231
cc_by_ip,0.00155408
countryLabel,0.00181712


In [28]:
display_df(df_clean,200)

,course_id,user_id,registered,viewed,explored,certified,completed,ip,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,un_major_region,un_economic_group,un_developing_nation,un_special_region,latitude,longitude,LoE,YoB,gender,grade,passing_grade,start_time,first_event,last_event,nevents,ndays_act,nplay_video,nchapters,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,roles,nprogcheck,nproblem_check,nforum_events,mode,is_active,cert_created_date,cert_modified_date,cert_status
1,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
2,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
3,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
4,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
5,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
6,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
7,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
8,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
9,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
10,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,


The first perceivable error in the dataset occurs under the completed field. It often has the IP in it. Therefore, one of the early fields is the source of the problem. Perhaps one field is missing. However we are finding it difficult to determine which one. Sometimes, `viewed` is empty, sometimes `explored` is. Often we see `registered` equal to False, which also makes no sense as in order to be a row in the dataset that user needs to be registered for the course. Initially we are thinking that whenever `registered` is False, that it should really be True and that is the source of the missing field and therefore the cause of the shift. However, we cant be certain. Sometimes `registered` is indeed True. `completed` is an important field for researchers of the edX dataset, so we most definitely must repair this column, and we can likely assume that one of the previous columns, probably `certified` contains the values for that column. But which column is missing? The four fields previous `completed` are all booleans, so it is difficult to identify where the problem originates. We know for sure that if the `ip` is in the completed column we can shift the datapoints starting in `completed` over to the right one. We have decided that the source of the shift is the `registered` column. It is the least important for researchers as it is redundant and should always be True. Therefore we start the shift over from there and fill the value with True.

We target the `completed` column for values that look like IP addresses. In this case, we are looking for values that would not be the acceptable values in the `completed` column. The `completed` column should have either True, False, or perhaps None or empty string - anything that is not these is likely an IP address. We then shift everything beginning at `registered` over as that is a reasonable assumption as to what the missing column is and we will then fill that column with Trues as everyone in the database is registered for the class. We do recognize that any of the other boolean fields could have been the source of the shift like `viewed`, `explored`, `certified`, or even `completed` (though we think this is less likely given these fields seem more important for analysis).

In [159]:
#gets rows that look like the IP is in the completed field and shifts them
df_shift = df_clean[~df_clean['completed'].isin(['True','False','', None])]\
[df_clean.columns[2:]].shift(1, axis=1)
df_shift.registered = 'True'

In [162]:
df_clean.loc[~df_clean['completed'].isin(['True','False','', None]),\
            df_clean.columns[2:]] = df_shift

We have successfully fixed the shifting in the dataset.

In [165]:
display_df(df_clean)

,course_id,user_id,registered,viewed,explored,certified,completed,ip,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,un_major_region,un_economic_group,un_developing_nation,un_special_region,latitude,longitude,LoE,YoB,gender,grade,passing_grade,start_time,first_event,last_event,nevents,ndays_act,nplay_video,nchapters,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,roles,nprogcheck,nproblem_check,nforum_events,mode,is_active,cert_created_date,cert_modified_date,cert_status
1,HarvardX/PH525.1x/1T2018,7940,True,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,
2,HarvardX/PH525.1x/1T2018,7940,True,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,
3,HarvardX/PH525.1x/1T2018,7940,True,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,
4,HarvardX/PH525.1x/1T2018,7940,True,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,
5,HarvardX/PH525.1x/1T2018,7940,True,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661482,HarvardX/CS50x/2014_T1,1934738,True,False,,False,False,71.41.229.179,US,United States,Americas,Dallas,TX,Texas,75201,Northern America,Developed regions,,,32.7831,-96.8067,p,1972,m,0.0,0.6,2013-07-09 01:46:41,,,,,,,,,,,,,Student,,,,honor,1,2014-06-06 15:08:24,2014-09-15 16:35:38,notpassing
661483,HarvardX/CS50x/2014_T1,1934738,True,False,,False,False,71.41.229.179,US,United States,Americas,Dallas,TX,Texas,75201,Northern America,Developed regions,,,32.7831,-96.8067,p,1972,m,0.0,0.6,2013-07-09 01:46:41,,,,,,,,,,,,,Student,,,,honor,1,2014-06-06 15:08:24,2014-09-15 16:35:38,notpassing
661484,HarvardX/CS50x/2014_T1,1934738,True,False,,False,False,71.41.229.179,US,United States,Americas,Dallas,TX,Texas,75201,Northern America,Developed regions,,,32.7831,-96.8067,p,1972,m,0.0,0.6,2013-07-09 01:46:41,,,,,,,,,,,,,Student,,,,honor,1,2014-06-06 15:08:24,2014-09-15 16:35:38,notpassing
661485,HarvardX/CS50x/2014_T1,1934738,True,False,,False,False,71.41.229.179,US,United States,Americas,Dallas,TX,Texas,75201,Northern America,Developed regions,,,32.7831,-96.8067,p,1972,m,0.0,0.6,2013-07-09 01:46:41,,,,,,,,,,,,,Student,,,,honor,1,2014-06-06 15:08:24,2014-09-15 16:35:38,notpassing


In [168]:
#garbage, will have to deal with
display_df(df_clean[df_clean['completed'].isnull()])

,course_id,user_id,registered,viewed,explored,certified,completed,ip,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,un_major_region,un_economic_group,un_developing_nation,un_special_region,latitude,longitude,LoE,YoB,gender,grade,passing_grade,start_time,first_event,last_event,nevents,ndays_act,nplay_video,nchapters,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,roles,nprogcheck,nproblem_check,nforum_events,mode,is_active,cert_created_date,cert_modified_date,cert_status
758,HarvardX/PH525.1x/1T2018,529339,True,True,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1035,HarvardX/PH525.1x/1T2018,866059,False,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3495,HarvardX/PH525.1x/1T2018,3524821,True,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3828,HarvardX/PH525.1x/1T2018,4030650,True,True,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4925,HarvardX/PH525.1x/1T2018,5498643,True,True,False,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653964,HarvardX/CS50x/2014_T1,1842651,True,True,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
656251,HarvardX/CS50x/2014_T1,1876271,True,False,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
657242,HarvardX/CS50x/2014_T1,1893442,True,True,False,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
658870,HarvardX/CS50x/2014_T1,1916566,True,False,,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


A quick find shows that certain students with failing grades are marked as `completed`. This could a sign of corruption. Or perhaps something else that we do not understand. We are considering adding a field `corrupt` that would mark the corrupt lines and marking rows like this as such.

In [84]:
display_df(df_clean[(df_clean['gender'] < df_clean['grade']) & (df_clean['certified']=="True")],200)

,course_id,user_id,registered,viewed,explored,certified,completed,ip,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,un_major_region,un_economic_group,un_developing_nation,un_special_region,latitude,longitude,LoE,YoB,gender,grade,passing_grade,start_time,first_event,last_event,nevents,ndays_act,nplay_video,nchapters,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,roles,nprogcheck,nproblem_check,nforum_events,mode,is_active,cert_created_date,cert_modified_date,cert_status
68230,HarvardX/HKS101A/2015T3,1592054,True,False,True,True,74.70.103.13,US,United States,Americas,Schenectady,NY,New York,12309,Northern America,Developed regions,,,42.7944,-73.8651,p,1984,m,0.0,0.7,2016-01-09 21:37:22,2016-01-09 00:00:00,2016-01-09 21:39:47.924000,13,1,4,2,,,,,,,Student,0,0,0,honor,1,2016-01-13 22:05:11,2016-01-27 14:55:39,downloadable,
72863,HarvardX/HKS101A/2015T3,4261498,False,,True,True,83.110.73.36,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,,Western Asia,Developing_Nations,,,24.4667,54.3667,m,1987,m,0.0,0.7,2016-01-08 15:40:45,,,,,,,,,,,,,Student,,,,honor,1,2016-01-13 22:04:41,2016-01-27 14:59:40,downloadable,
72864,HarvardX/HKS101A/2015T3,4261498,False,,True,True,83.110.73.36,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,,Western Asia,Developing_Nations,,,24.4667,54.3667,m,1987,m,0.0,0.7,2016-01-08 15:40:45,,,,,,,,,,,,,Student,,,,honor,1,2016-01-13 22:04:41,2016-01-27 14:59:40,downloadable,
72865,HarvardX/HKS101A/2015T3,4261498,False,,True,True,83.110.73.36,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,,Western Asia,Developing_Nations,,,24.4667,54.3667,m,1987,m,0.0,0.7,2016-01-08 15:40:45,,,,,,,,,,,,,Student,,,,honor,1,2016-01-13 22:04:41,2016-01-27 14:59:40,downloadable,
72866,HarvardX/HKS101A/2015T3,4261498,False,,True,True,83.110.73.36,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,,Western Asia,Developing_Nations,,,24.4667,54.3667,m,1987,m,0.0,0.7,2016-01-08 15:40:45,,,,,,,,,,,,,Student,,,,honor,1,2016-01-13 22:04:41,2016-01-27 14:59:40,downloadable,
72867,HarvardX/HKS101A/2015T3,4261498,False,,True,True,83.110.73.36,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,,Western Asia,Developing_Nations,,,24.4667,54.3667,m,1987,m,0.0,0.7,2016-01-08 15:40:45,,,,,,,,,,,,,Student,,,,honor,1,2016-01-13 22:04:41,2016-01-27 14:59:40,downloadable,
72868,HarvardX/HKS101A/2015T3,4261498,False,,True,True,83.110.73.36,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,,Western Asia,Developing_Nations,,,24.4667,54.3667,m,1987,m,0.0,0.7,2016-01-08 15:40:45,,,,,,,,,,,,,Student,,,,honor,1,2016-01-13 22:04:41,2016-01-27 14:59:40,downloadable,
72869,HarvardX/HKS101A/2015T3,4261498,False,,True,True,83.110.73.36,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,,Western Asia,Developing_Nations,,,24.4667,54.3667,m,1987,m,0.0,0.7,2016-01-08 15:40:45,,,,,,,,,,,,,Student,,,,honor,1,2016-01-13 22:04:41,2016-01-27 14:59:40,downloadable,
72870,HarvardX/HKS101A/2015T3,4261498,False,,True,True,83.110.73.36,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,,Western Asia,Developing_Nations,,,24.4667,54.3667,m,1987,m,0.0,0.7,2016-01-08 15:40:45,,,,,,,,,,,,,Student,,,,honor,1,2016-01-13 22:04:41,2016-01-27 14:59:40,downloadable,
72871,HarvardX/HKS101A/2015T3,4261498,False,,True,True,83.110.73.36,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,,Western Asia,Developing_Nations,,,24.4667,54.3667,m,1987,m,0.0,0.7,2016-01-08 15:40:45,,,,,,,,,,,,,Student,,,,honor,1,2016-01-13 22:04:41,2016-01-27 14:59:40,downloadable,


We definitely see that some rows are shifted. However before we deal with this, we would like to wrangle the data with the knowledge of the data type. For example, things that say False are not really the boolean value for false, but rather the string. We will try to correct this now.

In [ ]:
"""
Run through every data point, checking whether it could be a boolean value, an integer,
a float, or a datetime, and if not then leave it as a string. dt_format is the datetime
format it tries first, dt_format_catch is the alternative datetime format.
"""
def fix_types(df, dt_format, dt_format_catch):
    

In [ ]:
def check_types(df):
    

In [96]:
def is_ip(s):
    return (len(s.split('.'))==4)

In [38]:
df_clean['grade'].iloc[0]

'0.7'

In [50]:
df_clean['start_time'].iloc[100]

'2018-01-01 21:55:17.732324'

In [64]:
try:
    d = datetime.strptime(df_clean['passing_grade'].iloc[0], "%Y-%m-%d %H:%M:%S.%f")
except ValueError as v:
    d = datetime.strptime(df_clean['passing_grade'].iloc[0], "%Y-%m-%d %H:%M:%S")

In [63]:
d

datetime.datetime(2018, 4, 10, 8, 30, 28)